# Basic exemple

In [1]:
import numpy as np
from roseau.load_flow import *

In [2]:
ground = Ground()
pref = PotentialRef(ground)  # Set the ground to 0

# Create a voltage source
un = 400 / np.sqrt(3)
voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", n=4, ground=ground, voltages=voltages)

# Add a new LV bus and a load
load_bus = Bus(id="bus", n=4)
load = DeltaPowerLoad("load", load_bus, [100, 100, 100])

# Add a LV line between the voltage source and the bus
lc = LineCharacteristics("z", z_line=0.1 * np.eye(4, dtype=complex))
line = SimplifiedLine(id="line", n=4, bus1=vs, bus2=load_bus, line_characteristics=lc, length=2.0)

In [3]:
# Create the electrical network
en = ElectricalNetwork.from_element(vs)

In [4]:
# Solve the load flow
en.solve_load_flow(auth=("username", "dev password"))

1

In [5]:
print(load_bus.potentials)

[ 230.85347265+6.77541676e-22j -115.42673632-1.99924972e+02j -115.42673632+1.99924972e+02j    0.        +0.00000000e+00j] volt


In [6]:
line.currents

(array([ 0.43317514-3.38770838e-21j, -0.21658757-3.75140678e-01j,
        -0.21658757+3.75140678e-01j,  0.        +0.00000000e+00j]) <Unit('ampere')>,
 array([-0.43317514+3.38770838e-21j,  0.21658757+3.75140678e-01j,
         0.21658757-3.75140678e-01j, -0.        +0.00000000e+00j]) <Unit('ampere')>)

In [7]:
buses_results, branches_results, loads_results = en.results
buses_results.head()

potential
bus_id phase                        
vs     a      230.9401008+0.0000000j
       b     -115.470054-200.000000j
       c     -115.470054+200.000000j
       n      0.00000000+0.00000000j
bus    a      230.8534703+0.0000000j

In [8]:
branches_results.head()

current1            current2
branch_id phase                                        
line      a      0.433175-0.000000j -0.433175+0.000000j
          b     -0.216588-0.375141j  0.216588+0.375141j
          c     -0.216588+0.375141j  0.216588-0.375141j
          n      0.000000+0.000000j -0.000000+0.000000j

In [9]:
# Change the load to an unbalanced one and rerun the load flow
load.update_powers([200, 0, 0])
en.solve_load_flow(auth=("username", "dev password"))
print(load_bus.potentials)

[ 230.85346181-5.00250125e-02j -115.38340797-1.99949975e+02j -115.47005384+2.00000000e+02j    0.        +0.00000000e+00j] volt
